In [ ]:
# Question 1: Gaussian Discrmimant Analysis
!pip install -U pandas
!pip install -U numpy
!pip install -U plotly==5.10.0

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

x = np.loadtxt('../data/q4/q4x.dat')
y = np.loadtxt('../data/q4/q4y.dat', dtype=str)

y = np.reshape(y,(-1,1))

In [ ]:
# Normalize the data
x = (x - np.mean(x,axis=0).reshape(1,-1)) / np.std(x,axis=0).reshape(1,-1)

# Num of examples
m = x.shape[0]

print(x.mean(), x.std())


In [ ]:
#GDA theta parameters
y[y=='Alaska']=1
y[y=='Canada']=0
phi = np.sum([y=='1'])/m

i_x0=[i for i in range(len(y)) if y[i]=='0']
i_x1=[i for i in range(len(y)) if y[i]=='1']
m_x0 = (np.mean(x[i_x0], axis=0)).reshape(1,-1)
m_x1 = (np.mean(x[i_x1], axis=0)).reshape(1,-1)

# Cov functio  with each class cov
def cov(x, m_x0, m_x1, y):
    c=np.zeros((2,2))
    c0=np.zeros((2,2))
    c1=np.zeros((2,2))
    m = len(x)
    even, odd = 0, 0
    for i in range(m):
        if(y[i]=='1'):
            c1+=np.dot((x[i:i+1]-m_x1).T, (x[i:i+1]-m_x1))
            even+=1
        else:
            c0+=np.dot((x[i:i+1]-m_x0).T, (x[i:i+1]-m_x0))
            odd+=1
    c=(c0+c1)/m
    c0/=odd
    c1/=even
    return c, c0, c1

cov, cov_0, cov_1 = cov(x, m_x0, m_x1, y)
print("Probability PHI for class label='Alaska' is={}".format(phi))
print("Mean of the Distribution for label='Alaska' is={}".format(m_x1))
print("Mean of the Distribution for label='Canada' is={}".format(m_x0))
print("The value of the covariance matrix is = \n{}".format(cov))

In [ ]:
#Question 1.b - Plot the training data
fig = plt.figure(figsize=(15,10))
plt.scatter(x[i_x0][:,0], x[i_x0][:,1], c='r',label='Canada')
plt.scatter(x[i_x1][:,0], x[i_x1][:,1], c='b',label='Alaska')
plt.legend()
plt.savefig('q4_b.png', dpi=1000, bbox_inches='tight')
plt.show(block=False)

In [ ]:
#Question 4.c - Describe the equation of the boundary

#calculating linear model parameters
cov_inv = np.linalg.inv(cov)
c = (1/2)*((m_x1.dot(cov_inv)).dot(m_x1.T) - m_x0.dot(cov_inv).dot(m_x0.T))
m = cov_inv.dot((m_x1-m_x0).T)
x_val = np.array([np.min(x[:, 1] -1 ), np.max(x[:, 1] +1 )]).reshape(1,-1)
y = (-1./m[0:1]).dot(m[1:2].dot(x_val)) - c


fig = plt.figure(figsize=(15,10))
plt.scatter(x[i_x0][:,0], x[i_x0][:,1], c='r',label='Canada')
plt.scatter(x[i_x1][:,0], x[i_x1][:,1], c='b',label='Alaska')
plt.plot(x_val.ravel(), y.ravel(), c='k',label='Decision Boundary')
plt.legend()
plt.savefig('q4_c.png', dpi=1000, bbox_inches='tight')
plt.show(block=False)

In [ ]:
# Question 4.d - Report the values of the parameter estimates
import math
cov0_inv = np.linalg.inv(cov_0)
cov1_inv = np.linalg.inv(cov_1)

cov0_det = np.linalg.det(cov_0)
cov1_det = np.linalg.det(cov_1)
temp = math.log(phi/(1-phi))

a = cov1_inv - cov0_inv
b = 2* (cov1_inv.dot(m_x1.T) - cov0_inv.dot(m_x0.T))
d = m_x1.dot(cov1_inv).dot(m_x1.T) - m_x0.dot(cov0_inv).dot(m_x0.T)- math.log(cov0_det/cov1_det)

x1_val= np.linspace((np.min(x[:,1])-1), (np.max(x[:,1])+1), 100)
c1 = a[1,1]
x2_val = []
for i in range(len(x1_val)):
    temp2 = (a[0,0]*(x1_val[i]**2)) - (b[0,0]*x1_val[i]) + d[0,0]
    temp3 = ((a[0,1]+a[1,0])*x1_val[i]) - b[1,0]
    x2_val.append(np.roots([c1, temp3, temp2]))

y_0= [(x2_val[i][0]) for i in range(len(x2_val))]
y_1= [(x2_val[i][1]) for i in range(len(x2_val))]

print("The value of the covariance matrix for label='Alaska' is = \n{}".format(cov_1))
print("The value of the covariance matrix for label='Canada' is = \n{}".format(cov_0))
print("The value of the mean for label='Alaska' is = \n{}".format(m_x0))
print("The value of the mean for label='Canada' is = \n{}".format(m_x1))

In [ ]:
# Question 4.e - Describe the equation for the quadratic boundary

fig = plt.figure(figsize=(15,10))
plt.scatter(x[i_x0][:,0], x[i_x0][:,1], c='r',label='Canada')
plt.scatter(x[i_x1][:,0], x[i_x1][:,1], c='b',label='Alaska')
plt.plot(x_val.ravel(), y.ravel(), c='k',label='Decision Boundary')
plt.scatter(x1_val,y_0, c='k',s=4,label='Quadratic Decision Boundary')
plt.scatter(x1_val,y_1, c='k',s=4,label='Quadratic Decision Boundary')
plt.legend()
plt.savefig('q4_e.png', dpi=1000, bbox_inches='tight')
plt.show(block=False)